<a href="https://colab.research.google.com/github/JuanManiglia/Guia_Tensorflow_Keras/blob/main/Clase_07/clase_07_5_tabular_synthetic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Modulo 7 : GANs**
* Instructor: [Juan Maniglia](https://juanmaniglia.github.io)

In [1]:
try:
    from google.colab import drive
    COLAB = True
    print("Con Google CoLab")
    %tensorflow_version 2.x
except:
    print("Sin Google CoLab")
    COLAB = False

Note: using Google CoLab


# Modulo 7.5: GAN para la generación de datos sintéticos tabulares

Normalmente, las GAN se utilizan para generar imágenes. Sin embargo, también podemos generar datos tabulares a partir de una GAN. En esta parte, usaremos la utilidad Tabgan de Python para crear datos falsos a partir de datos tabulares. Específicamente, usaremos el conjunto de datos Auto MPG para entrenar un GAN para generar autos falsos.[Cite:ashrapov2020tabular](https://arxiv.org/pdf/2010.00638.pdf)


In [2]:
!wget https://raw.githubusercontent.com/Diyago/GAN-for-tabular-data/master/requirements.txt
!pip install -r requirements.txt
!pip install tabgan

--2021-12-29 13:26:26--  https://raw.githubusercontent.com/Diyago/GAN-for-tabular-data/master/requirements.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 197 [text/plain]
Saving to: ‘requirements.txt.7’

requirements.txt.7  100%[===================>]     197  --.-KB/s    in 0s      

2021-12-29 13:26:26 (7.26 MB/s) - ‘requirements.txt.7’ saved [197/197]



### Cargar los datos de Auto MPG y entrenar una red neuronal

Comenzaremos generando datos falsos para el conjunto de datos Auto MPG que hemos visto anteriormente. La biblioteca tabgan puede generar datos categóricos (textuales) y continuos (numéricos). Sin embargo, no puede generar datos no estructurados, como el nombre del automóvil. La GAN no puede replicar los nombres de los automóviles, como "AMC Rebel SST", porque cada fila tiene un nombre de automóvil diferente; es un valor textual pero no categórico.

El siguiente código es similar a lo que hemos visto antes. Cargamos el conjunto de datos de AutoMPG. La biblioteca tabgan requiere el dataset de Pandas para entrenar. Debido a esto, mantenemos los valores de Pandas y Numpy.

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
import pandas as pd
import io
import os
import requests
import numpy as np
from sklearn import metrics

df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/auto-mpg.csv", 
    na_values=['NA', '?'])

COLS_USED = ['cylinders', 'displacement', 'horsepower', 'weight', 
          'acceleration', 'year', 'origin','mpg']
COLS_TRAIN = ['cylinders', 'displacement', 'horsepower', 'weight', 
          'acceleration', 'year', 'origin']

df = df[COLS_USED]

# missing value
df['horsepower'] = df['horsepower'].fillna(df['horsepower'].median())


# Dividir en train and test
df_x_train, df_x_test, df_y_train, df_y_test = train_test_split(
    df.drop("mpg", axis=1),
    df["mpg"],
    test_size=0.20,
    #shuffle=False,
    random_state=42,
)

# Crear versiones de dataframe para GAN tabular
df_x_test, df_y_test = df_x_test.reset_index(drop=True), df_y_test.reset_index(drop=True)
df_y_train = pd.DataFrame(df_y_train)
df_y_test = pd.DataFrame(df_y_test)

# Pandas to Numpy
x_train = df_x_train.values
x_test = df_x_test.values
y_train = df_y_train.values
y_test = df_y_test.values

# construimos la red neuronal
model = Sequential()
model.add(Dense(50, input_dim=x_train.shape[1], activation='relu')) # Hidden 1
model.add(Dense(25, activation='relu')) # Hidden 2
model.add(Dense(12, activation='relu')) # Hidden 3
model.add(Dense(1)) # Output
model.compile(loss='mean_squared_error', optimizer='adam')

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, 
        patience=5, verbose=1, mode='auto',
        restore_best_weights=True)
model.fit(x_train,y_train,validation_data=(x_test,y_test),
        callbacks=[monitor], verbose=2,epochs=1000)

Epoch 1/1000
10/10 - 2s - loss: 86219.8438 - val_loss: 1912.8424 - 2s/epoch - 168ms/step
Epoch 2/1000
10/10 - 0s - loss: 14992.5957 - val_loss: 9762.6133 - 100ms/epoch - 10ms/step
Epoch 3/1000
10/10 - 0s - loss: 2588.4531 - val_loss: 1899.9395 - 104ms/epoch - 10ms/step
Epoch 4/1000
10/10 - 0s - loss: 2207.9507 - val_loss: 633.3818 - 106ms/epoch - 11ms/step
Epoch 5/1000
10/10 - 0s - loss: 457.9456 - val_loss: 784.1696 - 178ms/epoch - 18ms/step
Epoch 6/1000
10/10 - 0s - loss: 465.8445 - val_loss: 229.7097 - 97ms/epoch - 10ms/step
Epoch 7/1000
10/10 - 0s - loss: 293.9960 - val_loss: 258.0968 - 124ms/epoch - 12ms/step
Epoch 8/1000
10/10 - 0s - loss: 250.8663 - val_loss: 266.4640 - 244ms/epoch - 24ms/step
Epoch 9/1000
10/10 - 0s - loss: 233.4393 - val_loss: 224.1489 - 193ms/epoch - 19ms/step
Epoch 10/1000
10/10 - 0s - loss: 227.7182 - val_loss: 221.9316 - 188ms/epoch - 19ms/step
Epoch 11/1000
10/10 - 0s - loss: 218.8727 - val_loss: 228.2350 - 138ms/epoch - 14ms/step
Epoch 12/1000
10/10 - 0s

Ahora evaluamos la red neuronal entrenada para ver el RMSE. Usaremos esta red neuronal entrenada para comparar la precisión entre los datos originales y los datos generados por GAN. Más adelante veremos que tales comparaciones se pueden utilizar para la detección de anomalías. Podemos utilizar esta técnica para los sistemas de seguridad. Si una red neuronal entrenada con datos originales no funciona bien con datos nuevos, entonces los datos nuevos pueden ser sospechosos o falsos.

In [4]:
pred = model.predict(x_test)
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print("Final score (RMSE): {}".format(score))

Final score (RMSE): 5.337835604017036


### Entrenamiento de una GAN para Auto MPG

A continuación, entrenaremos la GAN para generar datos falsos a partir de los datos MPG originales. Hay bastantes opciones que puede ajustar para la GAN. El ejemplo que se presenta aquí utiliza la mayoría de los valores predeterminados. Estos son los hiperparámetros habituales que deben ajustarse para cualquier modelo y requieren algo de experimentación para obtener resultados óptimos.

In [5]:
from tabgan.sampler import OriginalGenerator, GANGenerator
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

#gen_x, gen_y = GANGenerator().generate_data_pipe(df_x_train, df_y_train, df_x_test, )

gen_x, gen_y = GANGenerator(gen_x_times=1.1, cat_cols=None,
           bot_filter_quantile=0.001, top_filter_quantile=0.999, is_post_process=True,
           adversarial_model_params={
               "metrics": "rmse", "max_depth": 2, "max_bin": 100, 
               "learning_rate": 0.02, "random_state": 42, "n_estimators": 500,
           }, pregeneration_frac=2, only_generated_data=False,
           gan_params = {"batch_size": 500, "patience": 25, "epochs" : 500,}).generate_data_pipe(df_x_train, df_y_train,
                                          df_x_test, deep_copy=True, only_adversarial=False, use_adversarial=True)



Fitting CTGAN transformers for each column:   0%|          | 0/8 [00:00<?, ?it/s]

Training CTGAN, epochs::   0%|          | 0/500 [00:00<?, ?it/s]

### Evaluación de los resultados de GAN

Si mostramos los resultados, podemos ver que los datos generados por GAN se parecen al original. Algunos valores, normalmente números enteros en los datos originales, tienen valores fraccionarios en los datos sintéticos.

In [6]:
gen_x

cylinders  displacement  horsepower  weight  acceleration  year  origin
0            8    130.377165   99.373601    3577     18.643748    80       3
1            7    113.257756   97.349653    3679     20.036553    70       3
2            8    125.571491   94.285124    4696     21.566364    75       3
3            7    126.892285   76.207964    3640     10.799983    76       1
4            8    140.185530   87.871277    3547     20.866782    72       2
..         ...           ...         ...     ...           ...   ...     ...
591          4     91.000000   68.000000    1985     16.000000    81       3
592          4    108.000000   70.000000    2245     16.900000    82       3
593          4    108.000000   75.000000    2350     16.800000    81       3
594          4     90.000000   75.000000    2108     15.500000    74       2
595          4     97.000000   75.000000    2171     16.000000    75       3

[596 rows x 7 columns]

Finalmente, presentamos los datos sintéticos a la red neuronal previamente entrenada para ver con qué precisión podemos predecir los objetivos sintéticos. Como podemos ver, se pierde algo de precisión de RMSE al pasar a datos sintéticos.

In [7]:
# Predict
pred = model.predict(gen_x.values)
score = np.sqrt(metrics.mean_squared_error(pred,gen_y.values))
print("Final score (RMSE): {}".format(score))

Final score (RMSE): 11.034968310336694
